# Data Science mit gebrauchten Autos von Willhaben

Hier befassen wir uns mit der Analyse des Datensatzes

In [15]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = None
pd.options.display.max_rows = None


In [29]:
data = pd.read_csv("./data/final.csv", sep=";")
df = data

C:\Users\Admin\AppData\Local\Temp\ipykernel_7464\1997134380.py:1: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("./data/final.csv", sep=";")



## 1. Feature Engineering

Das erste Ziel ist den Datensatz kennenzulernen. Dabei untersuchen wir die vorhandenen Features mittels deskriptiver Methoden. 

In [25]:
df.columns

Index(['id', 'heading', 'body_dyn', 'price', 'year_model', 'mileage', 'brand',
       'model', 'car_type', 'noofseats', 'engine_effect',
       'engine_fuel_resolved', 'transmission_resolved', 'warranty_resolved',
       'coordinates', 'postcode', 'state', 'district', 'location', 'orgname',
       'isprivate', 'equipment_resolved', 'ln_price', 'ln_mileage'],
      dtype='object')

Wir entfernen ein paar wenig versprechende Spalten:

In [30]:
df = df.drop(columns=["product_id", "published_string", "condition_resolved", "country", 
    "fnmmocount", "upselling_ad_searchresult", "no_of_owners", "address", "description"])

In [19]:
df.head(3)

,id,heading,body_dyn,price,year_model,mileage,brand,model,car_type,noofseats,engine_effect,engine_fuel_resolved,transmission_resolved,warranty_resolved,coordinates,postcode,state,district,location,orgname,isprivate,equipment_resolved
0,612174978,Skoda Octavia Combi Style TSI ACT,-,34990,2022,4500,Skoda,Octavia,Kombi / Family Van,5.0,110,Benzin,Schaltgetriebe,Nein,"47.05489,15.1365",8570,Steiermark,Voitsberg,Voitsberg,F. Fripertinger GmbH,0,Bordcomputer|elektr. Fensterheber|Sitzheizung|...
1,612174235,VW Golf R-Line TDI 4MOTION DSG,Extras: Coming-Home-Funktion LED-Rückleuchten ...,43990,2022,3000,VW,Golf,Limousine,5.0,110,Diesel,Automatik,Nein,"47.44651,15.30058",8605,Steiermark,Bruck-Mürzzuschlag,Kapfenberg,Porsche Kapfenberg,0,Leichtmetall-/Alufelgen|Anhängerkupplung|elekt...
2,612174149,Seat Tarraco Xcellence 2.0 TDI DSG 4Drive,Extras: Fahrwerksregelung elektronisch Automat...,38490,2019,40880,Seat,Tarraco,SUV / Geländewagen,7.0,110,Diesel,Automatik,Nein,"48.56585,13.98543",4150,Oberösterreich,Rohrbach,Rohrbach in Oberösterreich,Kneidinger Center GmbH,0,Leichtmetall-/Alufelgen|Bordcomputer|elektr. F...


Aus den ersten drei Zeilen sieht man, wo sich `pandas` schwer tut mit den Datentypen. Deshalb setzen wir die types selbst:

In [58]:
df = df.astype({
  "id": str, 
  "body_dyn": str,
  "engine_fuel_resolved": "category", 
  "transmission_resolved": "category",
  "postcode": str
  })

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220108 entries, 0 to 220107
Data columns (total 22 columns):
 #   Column                 Non-Null Count   Dtype   
---  ------                 --------------   -----   
 0   id                     220108 non-null  object  
 1   heading                220108 non-null  object  
 2   body_dyn               220064 non-null  object  
 3   price                  220108 non-null  int64   
 4   year_model             220108 non-null  int64   
 5   mileage                220108 non-null  int64   
 6   brand                  220108 non-null  object  
 7   model                  220108 non-null  object  
 8   car_type               220108 non-null  object  
 9   noofseats              217319 non-null  float64 
 10  engine_effect          220108 non-null  int64   
 11  engine_fuel_resolved   219656 non-null  category
 12  transmission_resolved  219898 non-null  category
 13  warranty_resolved      220108 non-null  object  
 14  coordinates         

Anschließend erstellen wir ein paar neue Features, die für später relevant sein können:

In [60]:
df["ln_price"] = df["price"].apply(lambda x: np.log(x) if x > 0 else 0)
df["ln_mileage"] = df["mileage"].apply(lambda x: np.log(x) if x > 0 else 0)
df["age"] = 2022 - df["year_model"]
df["has_warranty"] = df["warranty_resolved"].apply(lambda x: 1 if x in "Ja" else 0)

def check_defect(text: str):
  defect = 1 if sum([word in text.lower() for word in ["schaden", "unfall"]]) > 0 else 0
  return defect
df["defect"] = df["body_dyn"].apply(check_defect)

In [61]:
df.defect.describe()

count    220108.000000
mean          0.043142
std           0.203178
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: defect, dtype: float64